In [ ]:
pip install pydrive --upgrade

In [2]:
#importing neccessary python packages and mounting to drive

from PIL import Image
import numpy as np
import random
!pip install -U -q PyDrive
!pip install tabulate
from tabulate import tabulate
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


In [3]:
#adding the bmp files
img15 = np.array(Image.open(rb"/content/drive/MyDrive/2015.bmp"))

img20 = np.array(Image.open(rb"/content/drive/MyDrive/2020.bmp"))

In [4]:
#define the colours as numpy array

print(img15[678][366])
print(img20[10][10])
red = np.array([255,0,0])
dgreen = np.array([0,128,0])
blue = np.array([0,0,255])
yellow = np.array([255,255,0])
lgreen = np.array([0,255,0])
white = np.array([255,255,255])
colors = [red, yellow, blue, dgreen, lgreen]

[255   0   0]
[255 255 255]


In [5]:
#generating shape of images to get their height and width
print(img15.shape,img20.shape)
tm1520=np.full((6,6),0)

(2962, 1475, 3) (2962, 1475, 3)


In [6]:
#function to check particular pixel to compare the values taken from above
def color(x):
    if((x==blue).all()):
        return 0
    elif((x==red).all()):
        return 1    
    elif((x==lgreen).all()):
        return 2    
    elif((x==dgreen).all()):
        return 3  
    elif((x==yellow).all()):
        return 4  
    else:
        return 5
def colorRet(x):
    if(x==0):
        return blue #Water
    elif(x==1):
        return red #Urban
    elif(x==2):
        return lgreen #dense Vegetation
    elif(x==3):
        return dgreen #sparse Vegetation
    elif(x==4):
        return yellow #Barren lands
    elif(x==5):
        return white #Other


In [7]:
#Class for image, generates neighbor data of pixels, returns height length pixel and pxl color
#classmethod for loading from file path

class markovImg:
    def __init__(self, data):
        #self.path = filepath
        #self.npimg = np.array(Image.open(filepath))
        self.npimg = data
        self.imgH = self.npimg.shape[0] #height
        self.imgL = self.npimg.shape[1] #Length
        self.nbr = np.full((self.imgH, self.imgL), -1)
        #self.classes = [0,0,0,0,0]
        for i in range (self.imgH):
            for j in range (self.imgL):
                self.nbr[i][j] = self.neighbor(i, j)
        #        k = self.pxlc(i, j)
        #        if(k < 5):
        #          self.classes[k] += 1
        #self.total = sum(self.classes)
        #for i in self.classes:
        #    i = i / self.total
    @classmethod
    def link(self, filepath):
        return markovImg (np.array(Image.open(filepath)))
    def pxl(self, x, y):
        return self.npimg[x][y]
    def pxlc (self, x, y):
        return color(self.npimg[x][y])
    def h(self):
        return self.imgH
    def l(self):
        return self.imgL
    def neighbor(self, x, y):
        n=0
        try:
            nlist = [(self.pxlc(x+1, y+1)), (self.pxlc(x, y+1)), (self.pxlc(x-1, y+1)), (self.pxlc(x+1, y)), (self.pxlc(x-1, y)), (self.pxlc(x+1, y-1)), (self.pxlc(x, y-1)),  (self.pxlc(x-1, y-1))]
            n2 = [i for i in nlist if i>4]
            if(n2==[] and self.pxlc(x,y)<5):
                n = nlist.count(0) + 10 * nlist.count(1) + 100 * nlist.count(2) + 1000 * nlist.count(3) + 10000 * nlist.count(4) + 100000 * color(self.pxl(x, y))
                return n
            else:
                return -1
        except:

            return -1

In [8]:
#to check img dimensions are same 
#store each pixel changed from previous img and its neighbours
def CAgen(img1, img2, img3):
    if (img1.h() == img2.h() == img3.h() and img1.l() == img2.l() == img3.l()):
        occ = {}
        n2 = np.full((img1.h(), img1.l()), 0)
        for i in range (img1.h()):
            for j in range (img1.l()):
                if(img1.nbr[i][j] != -1 and img2.nbr[i][j] != -1):
                    k = img1.nbr[i][j]
                    k2 = img2.nbr[i][j]
                    if k in [*occ]:
                        if (img2.pxlc(i,j) < 5):
                            occ[k][img2.pxlc(i,j)] = occ[k][img2.pxlc(i,j)] + 1
                    else:
                        occ[k] = [0,0,0,0,0]
                        occ[k][img2.pxlc(i,j)] = occ[k][img2.pxlc(i,j)] + 1
                    if k2 in [*occ]:
                        if (img3.pxlc(i,j) < 5):
                            occ[k2][img3.pxlc(i,j)] = occ[k2][img3.pxlc(i,j)] + 1
                    else:
                        occ[k2] = [0,0,0,0,0]
                        if (img3.pxlc(i,j) < 5):
                            occ[k2][img3.pxlc(i,j)] = occ[k2][img3.pxlc(i,j)] + 1
        for i in occ.values():
            s = sum(i)
            for j in i:
                j = j
        for i in [*occ]:
            s = sum(occ[i])
            if (s > 1):
                for j in range (len(occ[i])):
                    occ[i][j] = occ[i][j] / s
                    if (j>0):
                        occ[i][j] += occ[i][j-1]
        return occ
def CApredict(occ, ipimg, filename = "newimg.bmp", oldname = "Image 1", newname = "Image 2", mode = 0):
    opimg = np.full((ipimg.h(), ipimg.l(), 3), 255)
    for i in range (ipimg.h()):
        for j in range (ipimg.l()):
            if (ipimg.pxlc(i,j)<5):
                p = ipimg.nbr[i][j]
                if (p in [*occ]):
                    #MAXIMUM mode=0
                    if(mode==0):
                        opimg[i][j] = colorRet (occ[ipimg.nbr[i][j]].index(max(occ[ipimg.nbr[i][j]])))
                    #RANDOM mode=1
                    if(mode==1):
                        x = float(random.randint(0, 1000000))/1000000                    
                        for k in range(5):
                            if(x<=occ[p][k]):
                                opimg[i][j] = colorRet(k)
                                break
                else:
                    opimg[i][j] = ipimg.pxl(i, j)
    opimgMarkov = markovImg(opimg)
    img = Image.fromarray(opimg.astype('uint8'), 'RGB')
    img.save(filename)
    img.show()
    c = compare (ipimg, opimgMarkov)
    row0 = ["Image", "Water", "Urban", "Sparse Vegetation", "Dense Vegetation", "Barren Land"]
    row1 = [oldname] + c[0]
    row2 = ["Change%", 0,0,0,0,0]
    row3 = [newname] + c[1]
    for i in range (5):
        row2[i+1] = (int(row3[i+1]) / int(row1[i+1]) * 100 - 100) 
    table = [row0, row1, row2, row3]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))
def compare (img1, img2): #
    if (img1.h() == img2.h() and img1.l() == img2.l()):
        c1 = [0,0,0,0,0]
        c2 = [0,0,0,0,0]
        for i in range (img1.h()):
            for j in range (img1.l()):
                if (img1.pxlc(i, j) < 5 and img2.pxlc (i, j) < 5):
                    c1 [img1.pxlc(i, j)] += 1
                    c2 [img2.pxlc(i, j)] += 1
        return [c1, c2]
def markovPredict(img1, img2, filename = "newimg.bmp"):
    if (img1.h() == img2.h() and img1.l() == img2.l()):
        m = np.full((5,5),0)
        m = m.astype(np.float32)
        for i in range (img1.h()):
            for j in range (img1.l()):                
                if (color(img1.pxl(i, j)) < 5 and  color(img2.pxl(i, j)) < 5):
                    m [color(img1.pxl(i, j))] [color(img2.pxl(i, j))] += 1
        rowsum = m.sum(axis=1)
        for i in range (5):
            for j in range (5):
                m[i][j] = m[i][j] / rowsum[i]
        for i in range (5):
            for j in range (4):
                m[i][j+1] += m[i][j]
        newimg = np.full((img1.h(), img1.l(), 3), 255, dtype = np.uint8)
        for i in range(img1.h()):
            for j in range(img1.l()):
                if (color(img1.pxl(i, j)) <= 4 and color(img2.pxl(i, j))<=4):
                    x = float(random.randint(0, 1000000))/1000000
                    y = color(img2.pxl(i, j))
                    for k in range(5):
                        if(x<=m[y][k]):
                            newimg[i][j] = colorRet(k)
                            break
        img = Image.fromarray(newimg, 'RGB')
        img.save(filename)
        img.show()

In [9]:
img2000 = markovImg.link(r"/content/drive/MyDrive/Maps/2000.bmp")
img2015 = markovImg.link(r"/content/drive/MyDrive/Maps/2015.bmp")
img2010 = markovImg.link(r"/content/drive/MyDrive/Maps/2010.bmp")
img2020 = markovImg.link(r"/content/drive/MyDrive/Maps/2020.bmp")

In [10]:
occ = CAgen(img2010, img2015, img2020)

In [ ]:
CApredict(occ, img2020, filename = "2025predicted.bmp", oldname = "2020", newname = "2025", mode = 1)

╒═════════╤═════════════╤═════════════╤═════════════════════╤════════════════════╤═══════════════╕
│ Image   │       Water │       Urban │   Sparse Vegetation │   Dense Vegetation │   Barren Land │
╞═════════╪═════════════╪═════════════╪═════════════════════╪════════════════════╪═══════════════╡
│ 2020    │ 144562      │ 1.05517e+06 │        296751       │       515734       │  289869       │
├─────────┼─────────────┼─────────────┼─────────────────────┼────────────────────┼───────────────┤
│ Change% │      1.6595 │ 2.90085     │            -5.04059 │           -2.07006 │      -2.54391 │
├─────────┼─────────────┼─────────────┼─────────────────────┼────────────────────┼───────────────┤
│ 2025    │ 146961      │ 1.08578e+06 │        281793       │       505058       │  282495       │
╘═════════╧═════════════╧═════════════╧═════════════════════╧════════════════════╧═══════════════╛
